In [6]:
import os

os.environ['ZENODO_ACCESS_TOKEN'] = 'OpQHit6HWvoYsxD5WryUuc3nrrKAlQKbOez7TwDzXUkJiNgIsKk1iEZ06vJy'
ACCESS_TOKEN = os.getenv('ZENODO_ACCESS_TOKEN')

In [7]:
# Restart the runtime before running this cell
import anndata
import numpy as np
import scanpy as sc
import scvi

/home/cane/.local/share/hatch/env/virtual/scvi-tools/eVVa01t5/scvi-tools/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/cane/.local/share/hatch/env/virtual/scvi-tools/eVVa01t5/scvi-tools/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/cane/.local/share/hatch/env/virtual/scvi-tools/eVVa01t5/scvi-tools/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/home/cane/.local/share/hatch/env/virtual/scvi-tools/eVVa01t5/scvi-tools/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing CSCDataset from `anndata.experimental` is deprecated. Import anndat

In [8]:
import gc
import os

In [9]:
import requests
res = requests.get("https://zenodo.org/api/records/7587774")
tissue_download_path = {ind['key'][3:-14]:ind for ind in res.json()['files']}

In [ ]:
tissue_download_path

We loop here through all tissues and train all 4 models on the input data.
First step is to download the data, then hvg gene filtering, training all 4 models, using minified version of scANVI and scVI and not storing the data for condSCVI and Stereoscope which don't support minified versions yet.
We use non-default parameters for scVI and scANVI to make those models ready for scArches after downloading the models. Additionally, we use 3 layers to account for stronger batch effect due to Smartseq2 and 10X.
Intestine is filtered out from preprocessing as it was an error while uploading and it contains 'Large_Intestine' data.

In [5]:
# Following parameters are specific to Tabula Sapiens dataset
labels_key='cell_ontology_class'
batch_key = 'donor_assay'

# Folder structure for files and models.
data_dir = 'downloaded_ts_data/'
model_dir = 'pretrained_model_final/'
unknown_celltype_label = 'unknown'

scvi_model_kwargs = {
            "dropout_rate": 0.05,
            "dispersion": "gene",
            "n_layers": 3,
            "n_latent": 20,
            "gene_likelihood": "nb",
            "use_batch_norm": "none",
            "use_layer_norm": "both",
            "encode_covariates": True,
        }

In [6]:
from lightning.pytorch.loggers.tensorboard import TensorBoardLogger
from pooch import retrieve

In [10]:
finished = [] # 'Tongue', 'Mammary', 'Salivary_Gland', 'Skin', 'Heart'

In [11]:
for tissue in tissue_download_path.keys():
    if tissue in finished:
        print(f'{tissue} already finished. Skipping recomputation!')
        continue
    refdata_zenodo = tissue_download_path[tissue]
    retrieve(
        url=tissue_download_path[tissue]['links']['self'],
        known_hash=tissue_download_path[tissue]['checksum'],
        fname=f'TS_{tissue}.h5ad',
        path=data_dir,
        processor=None,
    )
    # Download reference dataset
    output_fn = f"{data_dir}/TS_{tissue}.h5ad"
    adata = anndata.io.read_h5ad(output_fn)
    adata.var.drop('feature_name', axis=1, inplace=True)

    sc.pp.highly_variable_genes(
        adata,
        n_top_genes=3000,
        subset=True,
        flavor='seurat_v3',
        span=1.0,
        batch_key=batch_key,
    )

    tensorboard_scvi = TensorBoardLogger(save_dir=f"{model_dir}/{tissue}/scvi")
    tensorboard_scanvi = TensorBoardLogger(save_dir=f"{model_dir}/{tissue}/scanvi")
    tensorboard_condscvi = TensorBoardLogger(save_dir=f"{model_dir}/{tissue}/condscvi")
    tensorboard_stereoscope = TensorBoardLogger(save_dir=f"{model_dir}/{tissue}/stereoscope")

    # Train scVI
    scvi.model.SCVI.setup_anndata(
                adata,
                batch_key=batch_key,
                labels_key=labels_key
            )
    scvi_model = scvi.model.SCVI(adata, **scvi_model_kwargs)
    scvi_model.train(max_epochs=100, train_size=0.8,
                     plan_kwargs={"n_epochs_kl_warmup": 100},
                     #logger=tensorboard_scvi
                     )

    qzm, qzv = scvi_model.get_latent_representation(give_mean=False, return_dist=True)
    scvi_model.adata.obsm['X_latent_qzm'] = qzm
    scvi_model.adata.obsm['X_latent_qzv'] = qzv

    # Train scANVI
    scanvi_model = scvi.model.SCANVI.from_scvi_model(
                scvi_model,
                unlabeled_category=unknown_celltype_label
            )

    scanvi_model.train(
                max_epochs=20,
                n_samples_per_label=20,
                plan_kwargs={"n_epochs_kl_warmup": 10},
                logger=tensorboard_scanvi)

    qzm, qzv = scanvi_model.get_latent_representation(give_mean=False, return_dist=True)
    scanvi_model.adata.obsm['X_latent_qzm'] = qzm
    scanvi_model.adata.obsm['X_latent_qzv'] = qzv

    scvi_model.minify_adata()
    scvi_model.save(model_dir + f"{tissue}/scvi", overwrite=True, save_anndata=True)

    scanvi_model.minify_adata()
    scanvi_model.save(model_dir + f"{tissue}/scanvi", overwrite=True, save_anndata=True)

    # Train CondSCVI
    scvi.model.CondSCVI.setup_anndata(
        adata,
        labels_key=labels_key
    )
    condscvi_model = scvi.model.CondSCVI(
        adata,
        n_latent=5,
        n_layers=2,
        dropout_rate=0.05,
        weight_obs=False
    )

    condscvi_model.train(max_epochs=200, train_size=0.8, logger=tensorboard_condscvi)
    condscvi_model.save(model_dir + f"{tissue}/condscvi", overwrite=True, save_anndata=False)

    # Train Stereoscope
    scvi.external.RNAStereoscope.setup_anndata(
        adata,
        labels_key = labels_key)
    stereoscope_model = scvi.external.stereoscope.RNAStereoscope(
        adata
    )
    stereoscope_model.train(max_epochs=100, train_size=0.8, logger=tensorboard_condscvi)
    stereoscope_model.save(model_dir + f"{tissue}/stereoscope", overwrite=True, save_anndata=False)

    adata.write(model_dir + f"{tissue}/adata.h5ad")
    gc.collect()
    finished.append(tissue)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/cane/.local/share/hatch/env/virtual/scvi-tools/eVVa01t5/scvi-tools/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.
/home/cane/.local/share/hatch/env/virtual/scvi-tools/eVVa01t5/scvi-tools/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Epoch 100/100: 100%|██████████| 100/100 [01:10<00:00,  1.42it/s, v_num=1, train_loss_step=2e+3, train_loss_epoch=2.03e+3]   

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|██████████| 100/100 [01:10<00:00,  1.42it/s, v_num=1, train_loss_step=2e+3, train_loss_epoch=2.03e+3]
INFO     Training for 20 epochs.                                                                                   


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/cane/.local/share/hatch/env/virtual/scvi-tools/eVVa01t5/scvi-tools/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


...

Epoch 200/200: 100%|██████████| 200/200 [03:12<00:00,  1.04it/s, v_num=0, train_loss_step=866, train_loss_epoch=893]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/cane/.local/share/hatch/env/virtual/scvi-tools/eVVa01t5/scvi-tools/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.
/home/cane/.local/share/hatch/env/virtual/scvi-tools/eVVa01t5/scvi-tools/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.



Epoch 100/100: 100%|██████████| 100/100 [01:20<00:00,  1.21it/s, v_num=0, train_loss_step=1.25e+5, train_loss_epoch=1.58e+5]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|██████████| 100/100 [01:20<00:00,  1.24it/s, v_num=0, train_loss_step=1.25e+5, train_loss_epoch=1.58e+5]


In [12]:
headers = {"Content-Type": "application/json"}
params = {'access_token': ACCESS_TOKEN}

In [13]:
import requests
r = requests.get("https://zenodo.org/api/deposit/depositions/",
                 params={'access_token': ACCESS_TOKEN})
print(r.json())

{'created': '2024-12-06T06:00:40.840367+00:00', 'modified': '2024-12-06T06:00:42.076383+00:00', 'id': 14286626, 'conceptrecid': '7608634', 'doi': '10.5281/zenodo.14286626', 'conceptdoi': '10.5281/zenodo.7608634', 'doi_url': 'https://doi.org/10.5281/zenodo.14286626', 'metadata': {'title': 'Tabula sapiens scvi-tools models for scvi hub', 'doi': '10.5281/zenodo.14286626', 'publication_date': '2024-12-05', 'description': '<p>These are pre-trained models and AnnData datasets based on Tabula sapiens. Models were subsequentially uploaded to scvi-hub and this repository is there to restore the models on hugging face.</p>\n\n<p>Data is preprocessed data downloaded from&nbsp;<a href="https://cellxgene.cziscience.com/collections/e5f58829-1a66-40b5-a624-9046778e74f5">https://cellxgene.cziscience.com/collections/e5f58829-1a66-40b5-a624-9046778e74f5</a>. Please refer to their data usage guide before reusing the data.</p>', 'access_right': 'open', 'creators': [{'name': 'Can Ergen', 'affiliation': Non

In [24]:
import tarfile

In [30]:
for tissue in tissue_download_path.keys():
    path = f"{model_dir}/{tissue}"
    print(tissue, path)
    filename = f'TS_{tissue}_filtered.h5ad'

    if os.path.exists(f"{tissue}_pretrained_models.tar.gz"):
        continue

    with tarfile.open(f"{tissue}_pretrained_models.tar.gz", "w") as tar:
        for model in ['scvi', 'scanvi', 'condscvi', 'stereoscope']:
            print(model)
            tar.add(f"{model_dir}/{tissue}/{model}")
    with open(f"{tissue}_pretrained_models.tar.gz", "rb") as fp:
        r = requests.put(
            "%s/%s" % ('https://zenodo.org/api/files/1aa14714-ce25-489a-a095-f697064a56b5', f"{tissue}_pretrained_models.tar.gz"),
            data=fp,
            params=params,
        )
    with open(f"{model_dir}/{tissue}/adata.h5ad", "rb") as fp:
        r = requests.put(
            "%s/%s" % ('https://zenodo.org/api/files/1aa14714-ce25-489a-a095-f697064a56b5', f"{tissue}_training_data.h5ad"),
            data=fp,
            params=params,
        )

Tongue pretrained_model_final//Tongue
Skin pretrained_model_final//Skin
Mammary pretrained_model_final//Mammary
Salivary_Gland pretrained_model_final//Salivary_Gland
Heart pretrained_model_final//Heart
Lung pretrained_model_final//Lung
Bone_Marrow pretrained_model_final//Bone_Marrow
Lymph_Node pretrained_model_final//Lymph_Node
Small_Intestine pretrained_model_final//Small_Intestine
Prostate pretrained_model_final//Prostate
Large_Intestine pretrained_model_final//Large_Intestine
Fat pretrained_model_final//Fat
Spleen pretrained_model_final//Spleen
Bladder pretrained_model_final//Bladder
Liver pretrained_model_final//Liver
Pancreas pretrained_model_final//Pancreas
scvi
scanvi
condscvi
stereoscope
Vasculature pretrained_model_final//Vasculature
scvi
scanvi
condscvi
stereoscope
Blood pretrained_model_final//Blood
scvi
scanvi
condscvi
stereoscope
Uterus pretrained_model_final//Uterus
scvi
scanvi
condscvi
stereoscope
